In [1]:
%reset -f
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config Completer.use_jedi = False

In [2]:
import sys
sys.path.insert(0,"..")
from tqdm import tqdm
# from tqdm.notebook import tqdm as tqdm
import numpy as np
import pandas as pd
import torch
torch.manual_seed(42)
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
torchaudio.set_audio_backend('soundfile')
import os
import random
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import psutil
import requests
import seaborn as sns

from utils import *
from datasets import *
from models import *
from transforms import *
from profiler import *

Available workers: 16


In [3]:
maxpeaks = 15
minpeaks = 1
maxscore = 12
minscore = 3.5
selection = f"peaks>{minpeaks} & peaks<{maxpeaks} & score>{minscore} & score<={maxscore} & duration>100"

# Melanogaster and Suzukii main data

In [6]:
mel = WingbeatDatasetProfiler(dsname="/home/kalfasyan/data/wingbeats/wingbeats_20211129/wingbeats_correct_dates/Melanogaster_202111_RPi/a/", custom_label=[0], rpiformat=True, 
                              bandpass_low=1, bandpass_high=5000)
mel.df = mel.df.query(selection)
mel.df.shape

Found 232 in dataset: /home/kalfasyan/data/wingbeats/wingbeats_20211129/wingbeats_correct_dates/Melanogaster_202111_RPi/a/, and 1 label(s): ['wingbeats_correct_dates']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


(161, 13)

In [7]:
from scipy.io.wavfile import write
import soundfile

new_dir = f"{BASE_DATAPATH}/Melanogaster_Rpi_converted_wingbeats"
if not os.path.isdir(new_dir):
    os.makedirs(new_dir)
    
for fname in mel.df.x.tolist():
    sig, rate = open_wingbeat(fname, rate=True, rpiformat=True)
    filepath = os.path.join(new_dir, fname.split('/')[-1])
#     write(filename=filepath, rate=8000, data=sig.numpy())
    soundfile.write(filepath, sig.numpy(), 8000, subtype='PCM_16')

In [9]:
suz = WingbeatDatasetProfiler(dsname="/home/kalfasyan/data/wingbeats/Flytrap_rpi/Backup_rpi_freeflight_20210328/20210328/wingbeats", custom_label=[0], rpiformat=True, 
                              bandpass_low=1, bandpass_high=5000)
suz.df = suz.df.query(selection)
suz.df.shape

Found 166 in dataset: /home/kalfasyan/data/wingbeats/Flytrap_rpi/Backup_rpi_freeflight_20210328/20210328/wingbeats, and 1 label(s): ['Backup_rpi_freeflight_20210328']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


(128, 13)

In [10]:
from scipy.io.wavfile import write
import soundfile

new_dir = f"{BASE_DATAPATH}/Suzukii_Rpi_converted_wingbeats"
if not os.path.isdir(new_dir):
    os.makedirs(new_dir)
    
for fname in suz.df.x.tolist():
    sig, rate = open_wingbeat(fname, rate=True, rpiformat=True)
    filepath = os.path.join(new_dir, fname.split('/')[-1])
#     write(filename=filepath, rate=8000, data=sig.numpy())
    soundfile.write(filepath, sig.numpy(), 8000, subtype='PCM_16')

In [112]:
def inspect_file(path):
  print("-" * 10)
  print("Source:", path)
  print("-" * 10)
  print(f" - File size: {os.path.getsize(path)} bytes")
  print(f" - {torchaudio.info(path)}")

In [113]:
inspect_file(filepath)

----------
Source: /home/kalfasyan/data/wingbeats/Suzukii_Rpi_converted_wingbeats/20210326_003812_Tna_Hna.wav
----------
 - File size: 10044 bytes
 - AudioMetaData(sample_rate=8000, num_frames=5000, num_channels=1, bits_per_sample=16, encoding=PCM_S)


In [102]:
filepath = "/home/kalfasyan/data/wingbeats/Melanogaster_RL/X/D_20_10_09_20_52_59/F201009_205258_100_G_050_T_25.9_H_89.7.wav"

In [103]:
inspect_file(filepath)

----------
Source: /home/kalfasyan/data/wingbeats/Melanogaster_RL/X/D_20_10_09_20_52_59/F201009_205258_100_G_050_T_25.9_H_89.7.wav
----------
 - File size: 10044 bytes
 - AudioMetaData(sample_rate=8000, num_frames=5000, num_channels=1, bits_per_sample=16, encoding=PCM_S)


In [104]:
import soundfile

In [105]:
filepath = suz.df.x.iloc[0]

In [108]:
data,rate = soundfile.read(filepath)

In [109]:
soundfile.write("")

array([0.00198364, 0.00036621, 0.00039673, ..., 0.00045776, 0.00167847,
       0.00384521])

In [79]:
suz = WingbeatDatasetProfiler(dsname="/home/kalfasyan/data/wingbeats/Melanogaster_RL/Y", custom_label=[0], rpiformat=False)
suz.df = suz.df.query(selection)

Found 29002 in dataset: /home/kalfasyan/data/wingbeats/Melanogaster_RL/Y, and 1 label(s): ['D. melanogaster']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


In [83]:
plt.plot(suz.wbts[100][0].T)

In [12]:
sig.shape

torch.Size([240000000])

In [ ]:
plt.plot(sig)